<a href="https://colab.research.google.com/github/christianwarmuth/transformer_adapter_bias_evaluation/blob/main/colab/Adapter_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluating Adapter 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)
#drive.mount("/content/gdrive/MyDrive/master_hpi/NLP_Project/code/", force_remount=True)

Mounted at /content/gdrive/


In [4]:
import sys
sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/')



In [5]:
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [6]:
#!pip install transformers==3.0.2
!pip3 install -r /content/gdrive/MyDrive/master_hpi/NLP_Project/code/requirements.txt

     |████████████████████████████████| 133kB 13.3MB/s 
     |████████████████████████████████| 6.1MB 14.0MB/s 
     |████████████████████████████████| 163kB 41.3MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 552kB 35.4MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 296kB 35.4MB/s 
     |████████████████████████████████| 122kB 41.8MB/s 
     |████████████████████████████████| 512kB 33.4MB/s 
     |████████████████████████████████| 696kB 33.5MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 870kB 32.3MB/s 
     |████████████████████████████████| 1.0MB 28.0MB/s 
     |████████████████████████████████| 194kB 39.2MB/s 
     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 133kB 36.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0

In [7]:
#!pip install adapter-transformers
!pip install datasets
!pip install colorama

[autoreload of six failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: 'NoneType' object has no attribute 'cStringIO'
]


     |████████████████████████████████| 245kB 12.7MB/s 
     |████████████████████████████████| 245kB 22.9MB/s 
     |████████████████████████████████| 122kB 21.1MB/s 


In [8]:
import pandas as pd
from collections import defaultdict 
import dataloader

# Dataset Loading - Stereoset

In [9]:

print(path+"dev.json")

/content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json


In [10]:
stereoset = dataloader.StereoSet(path+"dev.json")

In [11]:
stereoset

In [12]:
intersentence_examples = stereoset.get_intersentence_examples() 
intrasentence_examples = stereoset.get_intrasentence_examples()

In [13]:
intrasentence_examples[1].sentences[1].gold_label

'anti-stereotype'

In [14]:
id2term = {}
id2gold = {}
id2score = {}
example2sent = {}
domain2example = {"intersentence": defaultdict(lambda: []), "intrasentence": defaultdict(lambda: [])}

for example in intrasentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intrasentence'][example.bias_type].append(example)

for example in intersentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intersentence'][example.bias_type].append(example)

In [15]:
# When predictions are already done
#for sent in self.predictions.get('intrasentence', []) + self.predictions.get('intersentence', []):
#            self.id2score[sent['id']] = sent['score']

In [16]:
domain2example

{'intersentence': defaultdict(<function __main__.<lambda>>,
             {'gender': [<dataloader.IntersentenceExample at 0x7f21aba05790>,
              'profession': [<dataloader.IntersentenceExample at 0x7f21aba09890>,
               ...],
              'race': [<dataloader.IntersentenceExample at 0x7f21aba02c90>,
               ...],
              'religion': [<dataloader.IntersentenceExample at 0x7f21ab81cf50>,
               <dataloader.IntersentenceExample at 0x7f21ab1aae10>]}),
 'intrasentence': defaultdict(<function __main__.<lambda>>,
             {'gender': [<dataloader.IntrasentenceExample at 0x7f21ac30f390>,
              'profession': [<dataloader.IntrasentenceExample at 0x7f21ac309310>,
               ...],
              'race': [<dataloader.IntrasentenceExample at 0x7f21ac309b50>,
               ...],
              'religion': [<dataloader.IntrasentenceExample at 0x7f21ac0b5d50>,
               <dataloader.IntrasentenceExample at 0x7f21aba64610>]})}

In [17]:
!python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [18]:
!pip3 install transformers==3

     |████████████████████████████████| 757kB 14.1MB/s 
     |████████████████████████████████| 3.0MB 33.2MB/s 


In [24]:
from eval_template import *
import dataloader


class Args:
  no_cuda = False
  input_file = '/content/dev.json'
  output_file = path+ 'output/predictions.json'
  max_seq_length = 128
  batch_size = 1

args=Args()

#class input_args:
#  def __init__(self, no_cuda, input_file, output_file, max_seq_length):
#    self.no_cuda = no_cuda
 #   self.input_file = input_file
 #   self.output_file = output_file
#    self.max_seq_length = max_seq_length

#args = input_args(False, "/content/dev.json", "/content/predictions.json", 128)
#args = {#
#    "no_cuda": False, 
#    "input_file": "/content/dev.json", 
#    "output_file":"/content/predictions.json"
#}

evaluator = BiasEvaluator(**vars(args))
results = evaluator.evaluate()

if args.output_file is not None:
  output_file = args.output_file
else:
  output_file = f"predictions_SentimentModel.json"

  output_file = os.path.join(args.output_dir, output_file)
with open(output_file, "w+") as f:
  json.dump(results, f, indent=2)


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/SentimentBert.pth
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
Maximum sequence length found: -inf
Number of parameters: 109,483,778
odict_keys(['module.bert.embeddings.word_embeddings.weight', 'module.bert.embeddings.position_embeddings.weight', 'module.bert.embeddings.token_type_embeddings.weight', 'module.bert.embeddings.LayerNorm.weight', 'module.bert.embeddings.LayerNorm.bias', 'module.bert.encoder.layer.0.attention.self.query.weight', 'module.bert.encoder.layer.0.attention.self.query.bias', 'module.bert.encoder.layer.0.attention.self.key.weight', 'module.bert.encoder.layer.

100%|██████████| 6369/6369 [02:27<00:00, 43.27it/s]



Evaluating bias on intrasentence tasks...
Arrived 0
Number of parameters: 109,483,778
odict_keys(['module.bert.embeddings.word_embeddings.weight', 'module.bert.embeddings.position_embeddings.weight', 'module.bert.embeddings.token_type_embeddings.weight', 'module.bert.embeddings.LayerNorm.weight', 'module.bert.embeddings.LayerNorm.bias', 'module.bert.encoder.layer.0.attention.self.query.weight', 'module.bert.encoder.layer.0.attention.self.query.bias', 'module.bert.encoder.layer.0.attention.self.key.weight', 'module.bert.encoder.layer.0.attention.self.key.bias', 'module.bert.encoder.layer.0.attention.self.value.weight', 'module.bert.encoder.layer.0.attention.self.value.bias', 'module.bert.encoder.layer.0.attention.output.dense.weight', 'module.bert.encoder.layer.0.attention.output.dense.bias', 'module.bert.encoder.layer.0.attention.output.LayerNorm.weight', 'module.bert.encoder.layer.0.attention.output.LayerNorm.bias', 'module.bert.encoder.layer.0.intermediate.dense.weight', 'module.ber

100%|██████████| 6318/6318 [02:29<00:00, 42.40it/s]


In [21]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


{}
